In [1]:
import torch
torch.cuda.is_available()

import moviepy.editor as mp

#filename = "sample"
filename = "sample_10min"

my_clip = mp.VideoFileClip(filename+".mkv")
my_clip.audio.write_audiofile(filename+".mp3")

MoviePy - Writing audio in sample_10min.mp3


MoviePy - Done.


In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset, Dataset
import ffmpeg
import os
print(os.getcwd())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

#dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
#sample = dataset[0]["audio"]
#result = pipe(sample)

#audio_dataset = Dataset.from_dict({"audio": ["sample.mp3"]})
#sample = audio_dataset[0]["audio"]

result = pipe(filename+".mp3")
#print(result)



/home/qbateux/Code/Auto_Subs


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [3]:
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-fr")

translation_timestamped = []

for chunk in result["chunks"]:
    translation_timestamped.append([chunk["timestamp"],  pipe(chunk["text"])[0]["translation_text"]])

In [13]:
import datetime
final = []
i = 1
for timestamp, text in translation_timestamped:
    if timestamp[0] is None or timestamp[1] is None:
        print("Faulty timestamp:", timestamp)
        print("text:", text)
        continue
    
    t1_str = str(datetime.timedelta(seconds=timestamp[0]))
    t2_str = str(datetime.timedelta(seconds=timestamp[1]))

    if "." in t1_str:
        t1_str = t1_str.split(".")[0] + "," + t1_str.split(".")[1][0:3]
    if "." in t2_str:
        t2_str = t2_str.split(".")[0] + "," + t2_str.split(".")[1][0:3]

    final.append(str(i))
    final.append(t1_str + " --> " + t2_str)
    final.append(text)
    final.append("")
    i+=1

import codecs
with codecs.open(filename+'.srt', 'w', 'utf-8') as the_file:
    for line in final:
        print("line:", line)        
        the_file.write(line+'\n')
        

Faulty timestamp: (594.62, None)
text: Et à chaque heure du déjeuner.
line: 1
line: 0:00:01 --> 0:00:03
line: Bonjour, je suis Ricky Gervais, avec moi, Stephen Merchant.
line: 
line: 2
line: 0:00:03 --> 0:00:04
line: Allô.
line: 
line: 3
line: 0:00:04 --> 0:00:08
line: Créateurs, écrivains, réalisateurs, vedettes de The Office.
line: 
line: 4
line: 0:00:08 --> 0:00:10
line: Si vous êtes fan de cette série, qui ne l'est pas ?
line: 
line: 5
line: 0:00:10 --> 0:00:12
line: Ouais, alors tu as une friandise à venir,
line: 
line: 6
line: 0:00:12 --> 0:00:16
line: Parce que c'est toute la première série...
line: 
line: 7
line: 0:00:16 --> 0:00:17
line: S'ils sont fans, Rick,
line: 
line: 8
line: 0:00:17 --> 0:00:19
line: Alors ils auront déjà acheté le DVD et ont déjà vu le spectacle.
line: 
line: 9
line: 0:00:19 --> 0:00:25
line: Attends, laisse-moi finir, c'est toute la première série, mais avec des morceaux supplémentaires.
line: 
line: 10
line: 0:00:25 --> 0:00:27
line: On a des intervie